In [2]:
!pip install gradio

In [10]:
import faiss
import numpy as np
import gradio as gr
import pickle
from openai import OpenAI
from credentials.keys import OPENAI_API_KEY

print(f"FAISS version : {faiss.__version__}")
print(f"FAISS sur CPU ou GPU ? {faiss.get_num_gpus()}")  # Si 0 → FAISS utilise uniquement le CPU

# Chargement de l'index FAISS et des métadonnées
index = faiss.read_index("faiss_index_openai.bin")
with open("metadata_openai.pkl", "rb") as f:
    metadata = pickle.load(f)

# Initialisation du client OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# Fonction pour générer l'URL externe d'une série
def generate_series_url(series_key, dataset_id):
    base_url = "https://webstat.banque-france.fr/fr/catalogue/"
    return f"{base_url}{dataset_id}/{series_key}"

# Fonction pour calculer la variance de l'embedding
def compute_variance(embedding):
    return np.var(embedding)

# Fonction pour calculer la taille de police en fonction de la distance
def compute_font_size(distance, min_dist=0.5, max_dist=1.5, font_min=10, font_max=24):
    if distance <= min_dist:
        return font_max
    elif distance >= max_dist:
        return font_min
    else:
        return font_max - ((distance - min_dist) / (max_dist - min_dist)) * (font_max - font_min)

# Fonction de recherche dans FAISS
def search_query(query_text, series_key):
    if not query_text:
        return "Veuillez entrer une requête."
    
    # Génération de l'embedding de la requête
    response = client.embeddings.create(input=query_text, model="text-embedding-3-small")
    query_embedding = np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)
    
    # Recherche dans FAISS
    k = 10  # Nombre de résultats à récupérer
    distances, indices = index.search(query_embedding, k)
    
    output = f"Phrase recherchée : {query_text}\n\n"

    metadata_keys = list(metadata.keys())  # Liste des clés disponibles

    for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
        if idx < len(metadata_keys):  # Vérifier si l'index est valide
            doc_id = metadata_keys[idx]  # Récupérer la bonne clé
            meta_info = metadata[doc_id]
            dataset_id = meta_info.get('dataset_id', 'inconnu')
            series_url = generate_series_url(doc_id, dataset_id)
            
            # Calcul de la taille de police en fonction de la distance
            font_size = compute_font_size(distance)
            output += (f"<br><span style='font-size:{font_size}px'>"
                       f"Résultat {i+1}: <a href=\"{series_url}\" target=\"_blank\">{doc_id}</a> (distance: {distance})<br>"
                       f"  - Dataset: {dataset_id}<br>"
                       f"  - Description dataset: {meta_info.get('description', '')}<br>"
                       f"  - Nom série: {meta_info.get('title', '')}<br>"
                       f"  - Périodicité série: {meta_info.get('periodicity', '')}<br><br></span>")

    return output

# Interface Gradio
demo = gr.Interface(
    fn=search_query,
    inputs=[
        gr.Textbox(label="Entrez votre requête"),
        gr.Textbox(label="Entrez une series_key spécifique (optionnel)")
    ],
    outputs=gr.HTML(label="Résultats de la recherche"),
    title="Recherche de series Webstat avec les embeddings OpenAI et FAISS"
)

demo.launch(share=True)



"""
IMPRESSIONNANT
j'ai un projet immobilier, quels sont les taux actuels ? MIR1.M.FR.B.A2C.O.R.A.2250U6.EUR.N
en finnois
Minulla on kiinteistöprojekti, mitkä ovat nykyiset korot?

quelle est la conjoncture dans le vêtement en France ? CONJ.M.N01.S.IN.000CB.CCTSM000.10 ou CONJ.M.N01.S.IN.000CB.PVTSM000.10
taux de change pays anglophones   ; EXR.D.USD.EUR.SP00.A ou EXR.M.USD.EUR.SP00.A
taux de change pays de l'est ; XR.D.EEK.EUR.SP00.A ou EXR.D.RUB.EUR.SP00.A
évolution mensuelle des faillites d'entreprises ; DIREN.M.FR.DE.UL.DF.25.N.ZZ.TT



quel est le taux d'intérêt maximal légal qu'un établissement de crédit peut appliquer lorsqu'il accorde un prêt immobilier ? MIR1.M.FR.R.A22FRF.W3.U.A.2254FR.EUR.N

LOGIQUEMENT CORRECT
quel est l'encours total de la monaie fiduciaire ? BSI.M.U2.N.U.L10.X.1.U2.2300.Z01.E
Comment est l'inflation en france ? ICP.M.FR.N.000000.4.ANR

MOYENS
combien de personnes sont surendettées ?

MAUVAIS
taux de changes devises quotidiennes
on pourrait ajouter des valeurs des dimensions
"""



FAISS version : 1.10.0
FAISS sur CPU ou GPU ? 0
* Running on local URL:  http://127.0.0.1:7888
* Running on public URL: https://4b46c5cc7994ead804.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


"\nIMPRESSIONNANT\nj'ai un projet immobilier, quels sont les taux actuels ? MIR1.M.FR.B.A2C.O.R.A.2250U6.EUR.N\nen finnois\nMinulla on kiinteistöprojekti, mitkä ovat nykyiset korot?\n\nquelle est la conjoncture dans le vêtement en France ? CONJ.M.N01.S.IN.000CB.CCTSM000.10 ou CONJ.M.N01.S.IN.000CB.PVTSM000.10\ntaux de change pays anglophones   ; EXR.D.USD.EUR.SP00.A ou EXR.M.USD.EUR.SP00.A\ntaux de change pays de l'est ; XR.D.EEK.EUR.SP00.A ou EXR.D.RUB.EUR.SP00.A\névolution mensuelle des faillites d'entreprises ; DIREN.M.FR.DE.UL.DF.25.N.ZZ.TT\n\n\n\nquel est le taux d'intérêt maximal légal qu'un établissement de crédit peut appliquer lorsqu'il accorde un prêt immobilier ? MIR1.M.FR.R.A22FRF.W3.U.A.2254FR.EUR.N\n\nLOGIQUEMENT CORRECT\nquel est l'encours total de la monaie fiduciaire ? BSI.M.U2.N.U.L10.X.1.U2.2300.Z01.E\nComment est l'inflation en france ? ICP.M.FR.N.000000.4.ANR\n\nMOYENS\ncombien de personnes sont surendettées ?\n\nMAUVAIS\ntaux de changes devises quotidiennes\non po